In [5]:
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn
from torchinfo import summary # with torchsummary we got a bug, consider installing torchinfo
from torch.utils.data import DataLoader, TensorDataset # lets us load data in batches
from tqdm.notebook import tqdm # with tqdm we got a bug

In [6]:
# wget the file from https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
with open("shakespeare.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus_length = len(corpus)
    
print("Loaded a corpus of {0} characters".format(corpus_length))

Loaded a corpus of 1115394 characters


In [7]:
# Get a unique identifier for each char in the corpus, 
# then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}
print("Our corpus contains {0} unique characters.".format(num_chars))
print(corpus[:100])


Our corpus contains 65 unique characters.
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
print(decoding)
print(encoding)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47,

## Many to One  approach

In [9]:
# chop up our data into X and y, slice into roughly 
# (num_chars / skip) overlapping 'sentences' of length 
# sentence_length, and encode the chars
sentence_length = 20
skip = 1
X_data = []
y_data = []
for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}"
      .format(num_sentences, sentence_length))

Sliced our corpus into 1115374 sentences of length 20


In [10]:
print(X_data[0])
print([decoding[idx] for idx in X_data[0]])
print(decoding[y_data[0]])

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56]
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B', 'e', 'f', 'o', 'r']
e


In [11]:
# One-hot encode the data.
X = F.one_hot(torch.tensor(X_data), num_classes=num_chars).to(torch.float)
y = torch.tensor(y_data) #No need to encode labels in one-hot with pytorch,
                         #crossEntropy loss needs just the indexes (not 0-1 values)


# Double check our vectorized data before we sink hours into fitting a model
print("Sanity check y. Dimension: {0} # Sentences: {1} Characters in corpus: {2}"
      .format(y.shape, num_sentences, len(chars)))
print("Sanity check X. Dimension: {0} Sentence length: {1}"
      .format(X.size(), sentence_length))

Sanity check y. Dimension: torch.Size([1115374]) # Sentences: 1115374 Characters in corpus: 65
Sanity check X. Dimension: torch.Size([1115374, 20, 65]) Sentence length: 20


In [12]:
# Let's define our model
        
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, num_classes):
        super().__init__()
        
        # Defining the number of h layers and the nodes in each layer
        self.layer_size = layer_size
        self.hidden_size = hidden_size
        
        # RNN layers
        self.rnn = nn.RNN(input_size, hidden_size, layer_size, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.rnn(x)
        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1]
        # Convert the final state to our desired output shape (batch_size, output_size)
        out = self.fc(out)
        
        return out
    

In [13]:
hidden_size = 256
seq_length = 20
num_classes = 65
layer_size = 1
batch_size= 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RNNModel(num_classes, hidden_size, layer_size, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

summary(model, input_size=(batch_size, seq_length, num_classes))

Layer (type:depth-idx)                   Output Shape              Param #
RNNModel                                 [128, 65]                 --
├─RNN: 1-1                               [128, 20, 256]            82,688
├─Linear: 1-2                            [128, 65]                 16,705
Total params: 99,393
Trainable params: 99,393
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 213.82
Input size (MB): 0.67
Forward/backward pass size (MB): 5.31
Params size (MB): 0.40
Estimated Total Size (MB): 6.37

In [14]:
model

RNNModel(
  (rnn): RNN(65, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=65, bias=True)
)

In [15]:
# Create dataloader from X,y tensors
dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [16]:
def train(epoch, model, data_loader, log_interval=200):
    model.train()
    total_train_loss = 0
    total_correct = 0 
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(data_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)
        # Backpropagate. Updates the gradients buffer on each parameter
        loss.backward()
        # Update weights
        optimizer.step()
        
        total_train_loss += loss.item()
        
        _, pred = torch.max(output, dim=1)

        total_correct += torch.sum(pred == target).item()
                  
        
        # if batch_idx % log_interval == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(data_loader.dataset),
        #         100. * batch_idx / len(data_loader), loss.data.item()))
    
    accuracy_train = total_correct / len(data_loader.dataset)

    
    total_train_loss = total_train_loss / len(data_loader)

    return {
        "loss": total_train_loss,
        "accuracy": accuracy_train,
    }
# train the model
##%%time

# Keep track of stats to plot them
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

epochs = 10
for epoch in range(1, epochs + 1):
    train_result = train(epoch, model, train_loader)
    train_losses.append(train_result["loss"])
    train_accuracies.append(train_result["accuracy"])
    

Train Epoch: 1 [0/1115374 (0%)]	Loss: 4.184679
Train Epoch: 1 [25600/1115374 (2%)]	Loss: 2.742207
Train Epoch: 1 [51200/1115374 (5%)]	Loss: 2.481631
Train Epoch: 1 [76800/1115374 (7%)]	Loss: 2.232582
Train Epoch: 1 [102400/1115374 (9%)]	Loss: 2.401128
Train Epoch: 1 [128000/1115374 (11%)]	Loss: 2.344580
Train Epoch: 1 [153600/1115374 (14%)]	Loss: 2.089907
Train Epoch: 1 [179200/1115374 (16%)]	Loss: 2.253219
Train Epoch: 1 [204800/1115374 (18%)]	Loss: 2.185755
Train Epoch: 1 [230400/1115374 (21%)]	Loss: 2.127522
Train Epoch: 1 [256000/1115374 (23%)]	Loss: 2.094757
Train Epoch: 1 [281600/1115374 (25%)]	Loss: 2.091164
Train Epoch: 1 [307200/1115374 (28%)]	Loss: 2.113344
Train Epoch: 1 [332800/1115374 (30%)]	Loss: 2.094473
Train Epoch: 1 [358400/1115374 (32%)]	Loss: 2.146134
Train Epoch: 1 [384000/1115374 (34%)]	Loss: 1.989364
Train Epoch: 1 [409600/1115374 (37%)]	Loss: 1.769158
Train Epoch: 1 [435200/1115374 (39%)]	Loss: 2.125119
Train Epoch: 1 [460800/1115374 (41%)]	Loss: 1.849709
Train 

KeyboardInterrupt: 

In [ ]:
@torch.inference_mode() 
def predict(model, data):
    # Put the model in eval mode, which disables training specific behaviour.
    model.eval()
    output = model(data)
    return output

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.legend()
plt.grid()

In [ ]:
def make_seed(seed_phrase=""):
        if seed_phrase:  # make sure the seed has the right length
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:            # sample randomly the seed from corpus
            seed = random.randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]
        return pattern
    
    

seed_pattern = make_seed("Once upon a time in ")
print("seed = " + seed_pattern)

encoded_text = torch.tensor([encoding[char] for char in seed_pattern])
encoded_text = F.one_hot(encoded_text, num_classes=num_chars).to(torch.float)
# Add a single batch dimension at the beginning
encoded_text = encoded_text.unsqueeze(0)
encoded_text = encoded_text.to(device)

generated_text = ""
for i in range(500):
    # predict() gives a tensor of shape (1, 65) 
    # with 1 being the size of the batch, for that we use [0] to get a vector
    output = predict(model, encoded_text)[0]
    # Convert the output to probabilities
    probs = torch.softmax(output, dim=-1)
    # Randomly choose from a multinomial distribution with the output probabilities
    # make the generation more diverse.
    prediction = torch.multinomial(probs, num_samples=1)
    generated_text += decoding[int(prediction)]
    
    # One hot encode the new (predicted) character
    next_char_encoded = F.one_hot(prediction, num_classes=num_chars)
    # Make sure it has a singular batch and seq_len dimension in order to concatenate them.
    next_char_encoded = next_char_encoded.view(1, 1, num_chars)
    # Remove first char and glue the predicted one to the end
    encoded_text = torch.cat((encoded_text[:, 1:], next_char_encoded), dim=1)
print(generated_text)

## Many to many approach

In [ ]:
# chop up our data into X and y, slice into roughly 
# (num_chars / skip) overlapping 'sentences' of length 
# sentence_length, and encode the chars
sentence_length = 20
skip = 1
X_data = []
y_data = []
for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i+1:i+1 + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append([encoding[char] for char in next_char])

num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}"
      .format(num_sentences, sentence_length))

In [ ]:
print(X_data[0])

In [ ]:
print([decoding[idx] for idx in X_data[0]])
print([decoding[idx] for idx in y_data[0]])


In [ ]:
# One-hot encode the data.
X = F.one_hot(torch.tensor(X_data), num_classes=num_chars).to(torch.float)
y = torch.tensor(y_data)

In [ ]:
# many2many model
class mmRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, num_classes):
        super().__init__()
        
        # Defining the number of h layers and the nodes in each layer
        self.layer_size = layer_size
        self.hidden_size = hidden_size
        # RNN layers
        self.rnn = nn.RNN(input_size, hidden_size, layer_size, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        #out = out[:, -1] # as you can see this line was deleted: This means instead of taking just the last hidden
        #state with out = out[:, -1] (size: batch_size x num_classes), we keep all the hidden states because out has 
        #a size of: batch_size x seq_len x num_classes (one hidden state per character in the sequence).

        
        #During inference, we take only the last hidden state
        #because other predictions are used only to train the model
        if not self.train:
            out = out[:, -1]
            
        # the Linear layer works automatically with one more dimension.
        out = self.fc(out)
        
        return out

In [ ]:
hidden_size = 256
seq_length = 20
num_classes = 65
layer_size = 1
batch_size= 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelmm = mmRNNModel(num_classes, hidden_size, layer_size, num_classes).to(device)
optimizer = torch.optim.Adam(modelmm.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

summary(modelmm, input_size=(batch_size, seq_length, num_classes))

In [ ]:
def train(epoch, model, data_loader, log_interval=200):
    model.train()
    total_train_loss = 0
    total_correct = 0 
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(tqdm(data_loader, desc=f"Training Epoch {epoch}")):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 

        output = model(data)
        
        #the only modification here consists in modifying the cross-entropy, because it takes inputs with 
        #dimensions: (N, C, d1, d2, ..., dk), that means the dimension of the classes is always the 2nd, 
        #and all the extra dimensions come at the end. You have to change your output to size:
        # batch_size x seq_len x num_classes -> batch_size x num_classes x seq_len. 
        #It's just a transpose of the loss criterion (output.transpose(1, 2), target).
        output = output.transpose(1, 2) 
        loss = criterion(output, target)

        loss.backward()

        optimizer.step()
        
        total_train_loss += loss.item()
        
        _, pred = torch.max(output, dim=1)

        total_correct += torch.sum(pred == target).item()
                  
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.data.item()))
    
    accuracy_train = total_correct / len(data_loader.dataset)

    
    total_train_loss = total_train_loss / len(data_loader)

    return {
        "loss": total_train_loss,
        "accuracy": accuracy_train,
    }

In [ ]:
@torch.inference_mode() 
def predict(model, data):
    # Put the model in eval mode
    model.eval()
    output = model(data)
    return output

In [ ]:
# Create dataloader from X,y tensors
datasett = TensorDataset(X, y)
train_loader = DataLoader(datasett, batch_size=batch_size, shuffle=True)

In [ ]:
# train the model
# Keep track of stats to plot them
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

epochs = 10
for epoch in range(1, epochs + 1):
    train_result = train(epoch, modelmm, train_loader)
    train_losses.append(train_result["loss"])
    train_accuracies.append(train_result["accuracy"])
    

plt.plot(train_losses, label='Training loss')
plt.legend()
plt.grid()

In [ ]:
def make_seed(seed_phrase=""):
        if seed_phrase:  
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:            
            seed = random.randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]
        return pattern

seed_pattern = make_seed("In the early morning, the flower is shining")

# To Do as an optimization: add function here that takes as parameters the seed_pattern and the model

encoded_text = torch.tensor([encoding[char] for char in seed_pattern])
encoded_text = F.one_hot(encoded_text, num_classes=num_chars).to(torch.float)
# Add a single batch dimension at the beginning
encoded_text = encoded_text.unsqueeze(0)
encoded_text = encoded_text.to(device)
torch.manual_seed(42)
generated_text = ""
for i in range(500):
    output = predict(modelmm, encoded_text)[0]
    # Convert the output to probabilities
    probs = torch.softmax(output[-1], dim=-1)
    # make the generation more diverse.
    prediction = torch.multinomial(probs, num_samples=1)

    generated_text += decoding[int(prediction)]
    # One hot encode the new (predicted) character
    next_char_encoded = F.one_hot(prediction, num_classes=num_chars)
    # Make sure it has a singular batch and seq_len dimension in order to concatenate them.
    next_char_encoded = next_char_encoded.view(1, 1, num_chars)
    # Remove first char and glue the predicted one to the end
    encoded_text = torch.cat((encoded_text[:, 1:], next_char_encoded), dim=1)
print(generated_text)